# Piperack Generator

## Module imports and connection to STAAD

In [ ]:
from base.helper.general import *
from base.geometry_base.rectangle import *
from base.staad_base.geometry import *
from base.structural_elements.beam import *
from base.structural_elements.column import *
from base.structural_elements.brace import *
from base.piperack.portal import *
from base.piperack.piperack import *
from base.piperack.tier import *
from base.staad_base.load import *
from base.staad_base.design import *
from base.staad_base.property import *
from output.md_output import *

from base.staad_base.optimise_member import *
from IPython.display import display, Markdown
from pyperclip import copy

openSTAAD,STAAD_objects = get_openSTAAD()

geometry = STAAD_objects.geometry
property = STAAD_objects.property
output = STAAD_objects.output
support = STAAD_objects.support
load = STAAD_objects.load

add_beams = add_beams(geometry=geometry)
assign_profile = assign_profile(property=property)
assign_specification = assign_specification(property=property)

## Inputs

### Steel

In [ ]:
base_point_of_first_portal = Point3D(0,0,0)
width_of_piperack = 8
portal_distances = [0,8,16,24,30,36,42]
column_distances = [0,width_of_piperack]
tier_elevations = [3,6,9,12,14]
long_beam_elevations = [4.5,7.5,10.5,tier_elevations[-1]]
braces_placement = [False,True,False,False,True,False]
brace_pattern = BracePattern.X_Pattern

### Concrete

In [ ]:
pedestal_height = 2
pedestal_offset = 1

## Generator Code

### Profiles

In [ ]:
concrete_profile_list = {
    PiperackMembers.Pedestals : Rectangle(1.2,0.8)
}

steel_profile_list = {
    PiperackMembers.Columns : 'ISMB600',
    PiperackMembers.TierBeams : 'ISMB500',
    PiperackMembers.LongitudinalBeams : 'ISMB600',
    PiperackMembers.PlanBracing : 'ISA100x100x6',
    PiperackMembers.VerticalBracing : 'ISA150x150x10',
    PiperackMembers.Stubs : 'ISMB500',
    PiperackMembers.IntermediateTransverseBeams : 'ISMB500',
    PiperackMembers.IntermediateLongitudinalBeams : 'ISMB400'
}

profile_ids = {}

for member_type,profile in concrete_profile_list.items():
    if ((isinstance(profile, str) and profile.isdigit()) or isinstance(profile, (int, float))):
        profile_ids[member_type] = int(profile)
    else:
        profile_ids[member_type] = create_concrete_beam_property(property,profile)

for member_type,profile in steel_profile_list.items():
    if ((isinstance(profile, str) and profile.isdigit()) or isinstance(profile, (int, float))):
        profile_ids[member_type] = int(profile)
    else:
        profile_ids[member_type] = create_steel_beam_property(property,10,profile,0,0,0)
        
display(Markdown(create_profile_markdown_table(concrete_profile_list, steel_profile_list, profile_ids)))

### Supports

In [ ]:
support_id = support.CreateSupportFixed()

### Specifications

#### Releases

In [ ]:
start_release_spec = property.CreateMemberReleaseSpec(0,set_DOFReleaseArray())
end_release_spec = property.CreateMemberReleaseSpec(1,set_DOFReleaseArray())
set_start_end_release = get_start_end_release_function(property=property,start_release_spec=start_release_spec,end_release_spec=end_release_spec)

#### Truss and Offset

In [ ]:
truss_spec = property.CreateMemberTrussSpec()
member_offset_spec = property.CreateMemberOffsetSpec(0,0,0,1,0)

display(Markdown(create_spec_markdown_table(start_release_spec,end_release_spec,truss_spec,member_offset_spec)))

## Member Creation

### Primary Members

In [ ]:
portals = []
longitudinal_beams = []
stubs = []
intermediate_transverse_beams = []
intermediate_long_beams = []
plan_braces = []
vertical_braces = []
portal_beam_ids = [] 
portal_column_ids = [] 
portal_pedestal_ids = [] 
support_node_ids = []
long_beam_ids = [] 
stub_ids = []
intermediate_transverse_ids = []
intermediate_long_ids = []
plan_brace_ids = []
vertical_brace_ids = []
portal_count = len(portal_distances)

# Create first portal
portal : PiperackPortal = PiperackPortal(base=base_point_of_first_portal) 
for column in column_distances:
    support_point = Point3D(column,0,0)-Point3D(0,pedestal_height,0)
    portal.add_pedestal(Column3D(base=support_point,height=pedestal_height))
    portal.add_column(Column3D(base=Point3D(column,0,0),height=tier_elevations[-1])) 
    
for tier in tier_elevations:
    for column_i in range(len(column_distances)-1):
        portal.add_beam(Beam3D(start=Point3D(column_distances[column_i],tier,0),end=Point3D(column_distances[column_i+1],tier,0))) 

# Create all portals by shifting the first portal
portals = [portal.shift(Point3D(0,0,dist)) for dist in portal_distances]

# Create longitudinal beams between portals
for portal_i in range(len(portals)-1):
    for column in column_distances:
        for long_beam in long_beam_elevations:
            long_beam_x = Beam3D(start=Point3D(column,long_beam,portals[portal_i].base.z),end=Point3D(column,long_beam,portals[portal_i+1].base.z))
            longitudinal_beams.append(long_beam_x)
    
# Add portal members to model and collect IDs
for portal in portals:
    portal_beam_ids = [*portal_beam_ids,*add_beams(portal.beams)]
    portal_column_ids = [*portal_column_ids,*add_beams(portal.columns)]
    portal_pedestal_ids = [*portal_pedestal_ids,*add_beams(portal.pedestals)]

# Assign profiles to portal members
portal_beam_assign = assign_profile(portal_beam_ids,profile_ids[PiperackMembers.TierBeams])
portal_column_assign = assign_profile(portal_column_ids,profile_ids[PiperackMembers.Columns])
portal_pedestal_assign = assign_profile(portal_pedestal_ids,profile_ids[PiperackMembers.Pedestals])

# Add longitudinal beams and assign properties
long_beam_ids = add_beams(longitudinal_beams)
long_release = set_start_end_release(long_beam_ids)
long_assign = assign_profile(long_beam_ids,profile_ids[PiperackMembers.LongitudinalBeams])

# Add support nodes
for portal in portals:
    for pedestal in portal.pedestals:
        support_node_ids.append(add_support_node(geometry,support,pedestal.start,support_id))

# Assign specifications
assign_specification(portal_pedestal_ids,member_offset_spec)

# Create comprehensive markdown table
markdown_output = "## Primary Members Created\n\n"
markdown_output += "| Member Types     | IDs                  | Profile Assignment     | Release Assignment     | Notes |\n"
markdown_output += "| ---              | ---                  | ---                    | ---                    | ---   |\n"
markdown_output += f"| Portal Columns   | {portal_column_ids}  | {portal_column_assign} | -                      | Vertical support members |\n"
markdown_output += f"| Portal Pedestals | {portal_pedestal_ids}| {portal_pedestal_assign}| -                     | Foundation connections |\n"
markdown_output += f"| Portal Beams     | {portal_beam_ids}    | {portal_beam_assign}   | -                      | Transverse tier beams |\n"
markdown_output += f"| Long Beams       | {long_beam_ids}      | {long_assign}          | {long_release}         | Longitudinal beams between portals |\n"
markdown_output += f"| Support Nodes    | {support_node_ids}   | -                      | -                      | Foundation support points |\n"

display(Markdown(markdown_output))

### Secondary Members

In [ ]:
# Group longitudinal beams by Y coordinate
long_beams_y = group_beams_by_y(longitudinal_beams)

# Create vertical braces    
if(brace_pattern == BracePattern.V_Pattern):
    previous_y = None
    for y_i,group_y in long_beams_y.items():
        base = portal.base.y if not previous_y else previous_y
        long_beams_sorted = beams_sorted_yxz(group_y)

        for beam_i in range(len(long_beams_sorted)-1):
            portal_i = beam_i%(len(portals)-1)
            if(braces_placement[portal_i]):
                
                long_beam_i : Beam3D = long_beams_sorted[beam_i]

                vb_1 = Beam3D(  start=Point3D(long_beam_i.start.x,base,long_beam_i.start.z),
                                end=long_beam_i.mid())
                vb_2 = Beam3D(  start=Point3D(long_beam_i.start.x,base,long_beam_i.end.z),
                                end=long_beam_i.mid())
                    
                vertical_braces.append(vb_1)
                vertical_braces.append(vb_2)

        previous_y = y_i
        
elif(brace_pattern == BracePattern.X_Pattern):
# Initialize variables
    sorted_y = sorted(long_beams_y.keys())  # Sort keys of long_beams_y
    vertical_braces = []  # Ensure vertical_braces is defined (assuming it's a list)

    for i, current_y_key in enumerate(sorted_y):
        group_y = long_beams_y[current_y_key]
        
        # Determine previous and next y-keys
        previous_y_key = sorted_y[i - 1] if i > 0 else portal.base.y
        next_y_key = sorted_y[i + 1] if i + 1 < len(sorted_y) else None
        
        # Process only even-indexed groups
        if i % 2 == 0:
            base_y = portal.base.y if previous_y_key is None else previous_y_key
            long_beams_sorted = beams_sorted_yxz(group_y)
            
            # Iterate through beams (excluding the last one)
            for beam_idx in range(len(long_beams_sorted) - 1):
                portal_idx = beam_idx % (len(portals) - 1)
                
                if braces_placement[portal_idx]:
                    current_beam = long_beams_sorted[beam_idx]
                    
                    # Create vertical braces from base
                    vb_1 = Beam3D(
                        start=Point3D(current_beam.start.x, base_y, current_beam.start.z),
                        end=current_beam.mid()
                    )
                    vb_2 = Beam3D(
                        start=Point3D(current_beam.start.x, base_y, current_beam.end.z),
                        end=current_beam.mid()
                    )
                    
                    vertical_braces.extend([vb_1, vb_2])
                    
                    # Add braces to next y-key if it exists
                    if next_y_key is not None:
                        vb_3 = Beam3D(
                            start=Point3D(current_beam.start.x, next_y_key, current_beam.start.z),
                            end=current_beam.mid()
                        )
                        vb_4 = Beam3D(
                            start=Point3D(current_beam.start.x, next_y_key, current_beam.end.z),
                            end=current_beam.mid()
                        )
                        
                        vertical_braces.extend([vb_3, vb_4])

# Add vertical braces and assign properties
vertical_brace_ids = add_beams(vertical_braces)
vertical_brace_spec = assign_specification(vertical_brace_ids,truss_spec)
vertical_brace_assign = assign_profile(vertical_brace_ids,profile_ids[PiperackMembers.VerticalBracing])

# Create stub beams between adjacent longitudinal beams
create_stub = lambda beamA,beamB : Beam3D(start=beamA.mid(),end=beamB.mid())

for long_beam_i in range(len(longitudinal_beams)-1):
    long_ii:Beam3D = longitudinal_beams[long_beam_i]
    long_ij:Beam3D = longitudinal_beams[long_beam_i+1]
    if(long_ii.start.eq_x(long_ij.start) and long_ii.end.eq_z(long_ij.end)):
        stubs.append(create_stub(long_ii,long_ij))

stubs = beams_sorted_yzx(stubs)

# Create intermediate transverse beams from stubs
for stub_i in range(0, len(stubs), 2):
    if stub_i + 1 < len(stubs):  
        stub1, stub2 = stubs[stub_i], stubs[stub_i + 1]
        stub1_mid,stub2_mid = stub1.mid(),stub2.mid()
        intermediate_transverse_beams.append(Beam3D(start=stub1_mid,end=stub2_mid)\
                                                .shift_to_y(closest_to(tier_elevations,stub1_mid.mid(stub2_mid).y)))

# Add stubs and assign properties
stub_ids = add_beams(stubs)
stub_assign = assign_profile(stub_ids,profile_ids[PiperackMembers.Stubs])
stub_release = set_start_end_release(stub_ids)
    
# Add intermediate transverse beams and assign properties
intermediate_transverse_ids = add_beams(intermediate_transverse_beams)
inter_trans_assign = assign_profile(intermediate_transverse_ids,profile_ids[PiperackMembers.IntermediateTransverseBeams])
inter_trans_release = set_start_end_release(intermediate_transverse_ids)

# Create intermediate longitudinal beams and plan braces from portal beams
portal_beams_y = group_beams_by_y([beam for portal in portals for beam in portal.beams])
for group_y in portal_beams_y.values():
    sorted_beams = beams_sorted_yzx(group_y)
    for beam_i in range(len(sorted_beams)-1):
        beam1 : Beam3D = sorted_beams[beam_i]
        beam2 : Beam3D = sorted_beams[beam_i+1]
        intermediate_long_beams.append(Beam3D(start=beam1.mid(),end=beam2.mid()))

        if(braces_placement[beam_i]):
            mid_pt = beam1.start.mid(beam2.end)
            for pt_x in [beam1.start,beam1.end,beam2.start,beam2.end]:
                plan_braces.append(Beam3D(start=pt_x,end=mid_pt))

# Add intermediate longitudinal beams and assign properties
intermediate_long_ids = add_beams(intermediate_long_beams)
inter_long_assign = assign_profile(intermediate_long_ids,profile_ids[PiperackMembers.IntermediateLongitudinalBeams])
inter_long_release = set_start_end_release(intermediate_long_ids)

# Add plan braces and assign properties
plan_brace_ids = add_beams(plan_braces)
plan_brace_spec = assign_specification(plan_brace_ids,truss_spec)
plan_brace_assign = assign_profile(plan_brace_ids,profile_ids[PiperackMembers.PlanBracing])

# Create comprehensive markdown table for secondary members
markdown_output = "## Secondary Members Created\n\n"
markdown_output += "| Member Types              | IDs                        | Profile Assignment      | Release Assignment      | Specification           | Function |\n"
markdown_output += "| ---                       | ---                        | ---                     | ---                     | ---                     | ---      |\n"
markdown_output += f"| Stubs                    | {stub_ids}                 | {stub_assign}          | {stub_release}          | -                       | Pipe support connections |\n"
markdown_output += f"| Intermediate Trans Beams | {intermediate_transverse_ids} | {inter_trans_assign} | {inter_trans_release}   | -                       | Secondary transverse support |\n"
markdown_output += f"| Intermediate Long Beams  | {intermediate_long_ids}    | {inter_long_assign}    | {inter_long_release}    | -                       | Secondary longitudinal support |\n"
markdown_output += f"| Plan Braces              | {plan_brace_ids}           | {plan_brace_assign}    | -                       | {plan_brace_spec}       | Horizontal stability bracing |\n"
markdown_output += f"| Vertical Braces          | {vertical_brace_ids}       | {vertical_brace_assign}| -                       | {vertical_brace_spec}   | Vertical stability bracing |\n"

# Add analysis summary
markdown_output += "\n## Secondary Members Analysis\n\n"
markdown_output += f"- **Bracing Pattern**: {brace_pattern}\n"
markdown_output += f"- **Total Stubs**: {len(stub_ids)} (connecting longitudinal beam pairs)\n"
markdown_output += f"- **Total Intermediate Transverse**: {len(intermediate_transverse_ids)} (from stub midpoints)\n"
markdown_output += f"- **Total Intermediate Longitudinal**: {len(intermediate_long_ids)} (from portal beam midpoints)\n"
markdown_output += f"- **Total Plan Braces**: {len(plan_brace_ids)} (horizontal X-bracing)\n"
markdown_output += f"- **Total Vertical Braces**: {len(vertical_brace_ids)} (vertical V-bracing)\n"

display(Markdown(markdown_output))

## Loads

### Initialization


In [ ]:
add_conc_forces_to_members = add_conc_forces_to_members(load)
add_uniform_forces_to_members = add_uniform_forces_to_members(load)

### Member Identification

In [ ]:
nodes = get_node_incidences(geometry=geometry)
beam_objects:dict = get_beam_objects(geometry=geometry,property=None,nodes=nodes)
beam_nos:list = list(beam_objects.keys())

### Portal Beams

### Intermediate Transverse Beams

### Columns